In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import re
%matplotlib inline

## Data Cleaning and Exploration

In [3]:
cd /Users/Aaron/Desktop/Academic/BIA652\ Mulitivariate\ Analysis/Proj/Data

/Users/Aaron/Desktop/Academic/BIA652 Mulitivariate Analysis/Proj/Data


In [4]:
lst = pd.read_csv('London_listings.csv')
lst.shape

(42646, 95)

In [5]:
records = lst.shape[1]
features = lst.shape[0]

In [6]:
useful_col = ['price', 'id', 'name', 'host_id', 'host_name', 'summary', 'space' ,'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified',   'neighbourhood_cleansed',  'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities',  'latitude', 'longitude', 'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights', 'number_of_reviews', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'requires_license', 'license', 'jurisdiction_names', 'instant_bookable', 'cancellation_policy', 'require_guest_profile_picture', 'require_guest_phone_verification', 'calculated_host_listings_count', 'reviews_per_month']

In [7]:
lst_n = lst[useful_col]
lst_n.shape

(42646, 61)

In [8]:
Y = lst_n['price'].astype(str).str.strip('$')

In [9]:
## Convert target variable 'price' to standard float
Price = np.zeros(len(Y))
for i in range(len(Y)):
    try:
        Price[i] = np.float64(Y[i]) 
    except ValueError:
        Price[i] = np.float64(Y[i].replace(',',''))  

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
lst_n['Price'] = Price
lst_n['Price'].describe()

count    42646.000000
mean        96.753435
std        129.372547
min          7.000000
25%         45.000000
50%         70.000000
75%        119.000000
max       7001.000000
Name: Price, dtype: float64

In [12]:
lst_n[:1000].to_csv('sample_2.csv') ## write a sample data

In [13]:
response_time_map = {'within an hour': 0, 'within a few hours': 1, 'within a day': 2, 'a few days or more':3, np.nan: 4}

In [14]:
lst_n['n_host_response_time'] = lst_n['host_response_time'].map(response_time_map)
lst_n['n_host_response_time'].value_counts()

0    14227
1    10858
4     9516
2     7357
3      688
Name: n_host_response_time, dtype: int64

In [15]:
lst_n['host_response_rate'] = lst_n['host_response_rate'].fillna(0)
n_host_response_rate = lst_n['host_response_rate'].astype(str).str.strip('%')

In [16]:
## Convert variable 'n_host_response_rate' to standard float
nhrr = np.zeros(records)
for i in range(records):
    try:
        nhrr[i] = np.float64(n_host_response_rate[i])/ 100 
    except ValueError:
        pass 

In [17]:
lst_n['n_host_is_superhost'] = lst_n['host_is_superhost'].map({'f':0, 't':1})
lst_n['n_host_is_superhost'].value_counts()

0.0    40082
1.0     2553
Name: n_host_is_superhost, dtype: int64

In [18]:
def get_unique(variable):
    uniques = []
    for ele in lst_n[variable]:
        uniques.extend(ele.split(',').strip(''))
    return set(uniques)

In [19]:
lst_n['n_host_has_profile_pic'] = lst_n['host_has_profile_pic'].map({'f':0, 't':1})
lst_n['n_host_has_profile_pic'].value_counts()

1.0    42481
0.0      154
Name: n_host_has_profile_pic, dtype: int64

In [20]:
lst_n['n_host_identity_verified'] = lst_n['host_identity_verified'].map({'f':0, 't':1})
lst_n['n_host_identity_verified'].value_counts()

1.0    25939
0.0    16696
Name: n_host_identity_verified, dtype: int64

In [21]:
neighbor_dummies = pd.get_dummies(lst_n['neighbourhood_cleansed'], prefix = 'neighbourhood')
lst_n = pd.concat([lst_n, neighbor_dummies], axis = 1)

In [22]:
roomtype_dummies = pd.get_dummies(lst_n['room_type'], prefix = 'room_type')
lst_n = pd.concat([lst_n, roomtype_dummies], axis = 1)

In [23]:
bedtype_dummies = pd.get_dummies(lst_n['bed_type'], prefix = 'bed_type')
lst_n = pd.concat([lst_n, bedtype_dummies], axis = 1)

In [24]:
lst_n['security_deposit'] = lst_n['security_deposit'].fillna(0)
lst_n['cleaning_fee'] = lst_n['cleaning_fee'].fillna(0)

In [25]:
lst_n['n_instant_bookable'] = lst_n['instant_bookable'].map({'f':0, 't':1})
lst_n['n_instant_bookable'].value_counts()

0    35506
1     7140
Name: n_instant_bookable, dtype: int64

In [26]:
lst_n['cancellation_policy'] = lst_n['cancellation_policy'].map(
    {'strict': 1, 'super_strict_60':1, 'super_strict_30':1, 'no_refunds':1, 'moderate':2, 'flexible':3})

In [37]:
from datetime import datetime as dt

In [28]:
lst_n['require_guest_profile_picture'] =  lst_n['require_guest_profile_picture'].map({'f':0, 't':1})
lst_n['require_guest_phone_verification'] =  lst_n['require_guest_phone_verification'].map({'f':0, 't':1})

In [29]:
lst_n['reviews_per_month'] = lst_n['reviews_per_month'].fillna(0)

In [40]:
time1 = lst_n['host_since'][0]

In [41]:
time1

'2014-09-28'

In [46]:
time2 = dt.strptime(time1, '%Y-%m-%d')

In [44]:
time2

datetime.datetime(2014, 9, 28, 0, 0)

In [48]:
time3 = dt(2016, 6, 14)

In [49]:
time3

datetime.datetime(2016, 6, 14, 0, 0)

In [52]:
(time3 - time2).days

625

In [56]:
host_since_dt = [dt.strptime(t, '%Y-%m-%d') for t in lst_n['host_since'] ]

TypeError: strptime() argument 1 must be str, not int

In [55]:
lst_n['host_since'] = lst_n['host_since'].fillna(0)

In [57]:
for (ind, item) in enumerate(lst_n['host_since']):
    if isinstance(item, float):
        print(ind, item)
    else:
        pass

In [58]:
a = 3.0
isinstance(a, float)

True